In [19]:
import tensorflow as tf
import numpy as np
import pandas as pd
#import tensorflow.contrib.eager as tfe
#tf.enable_eager_execution()

In [20]:
def trainDataLoad(local=False,market=True,news=True,sample=False):
    try:
        from kaggle.competitions import twosigmanews

        if(not local):
            env = twosigmanews.make_env()
        (market_df, news_df) = env.get_training_data()

        print('Data fetched from kaggle with {} rows of market data and {} rows of news data'.format(market_df.shape, news_df.shape))
    except:
        filename=['marketdata_sample.csv','news_sample.csv']
        if(not sample):
            filename=['market_train.csv','news_train.csv']
        print('failed to load data from kaggle, loading data from local directory.')
        if(market):
            market_df=pd.read_csv('./sampleData/'+filename[0])
        if(news):
            news_df=pd.read_csv('./sampleData/'+filename[1])
        print('Train data loaded!')
    if(market & (not news)):
        return market_df
    if(news & (not market)):
        return news_df
    return (market_df,news_df)

In [21]:
def timeCut(df,time, replace=True):
    '''
    df: dataFrame with attribute time in datatime64 format
    time: a time in string
    return df slice cutting off the time before the time provided
    '''
    df.time=pd.to_datetime(df.time)
    time=pd.Timestamp(time)
    df_slice = df[df.time>time]
    if replace:
        df=df_slice
    return df_slice

def formatCodeSet(df,field):
    '''
    df:dataframe
    field:field name of the code in the form string in set format
    return the field formatted into array
    '''
    return df[field].str.findall(f"'([\w\./]+)'")

# Embeddings

### Example for embedding lookups

Embedding lookup is a matrix lookup. The parameters input is a matrix where each row is an item. The input is a row index query. Upon execution, a matrix will be constructed according to input row index.

In [22]:
# 定义一个未知变量input_ids用于存储索引
input_ids = tf.placeholder(dtype=tf.int32, shape=[None])

# 定义一个已知变量embedding，是一个5*5的对角矩阵
# embedding = tf.Variable(np.identity(5, dtype=np.int32))

# 或者随机一个矩阵
embedding = np.asarray([[0.1, 0.2, 0.3], [1.1, 1.2, 1.3], [2.1, 2.2, 2.3], [3.1, 3.2, 3.3], [4.1, 4.2, 4.3]])

# 根据input_ids中的id，查找embedding中对应的元素
input_embedding = tf.nn.embedding_lookup(embedding, input_ids)

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

print(sess.run(input_embedding, feed_dict={input_ids: [1, 2, 3, 0, 3, 2, 1]}))

AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [5]:
market_train_df, news_train_df=trainDataLoad(sample=False)

failed to load data from kaggle, loading data from local directory.
Train data loaded!


To apply the embedding, we need to include all assets into the universe and at the same time need to take care of assets that are not included in current universe but may appear in the future. First we need to work out how many unique assets to be embedded.

In [6]:
# unique market assets in the market training data
market_assets=market_train_df.assetCode.unique()

In [7]:
#unique assets mentioned in the news training data
news_train_df['assetCodes'] = formatCodeSet(news_train_df,'assetCodes')
news_assets=np.unique([item for sublist in news_train_df.assetCodes.tolist() for item in sublist])
assets=np.unique(np.concatenate((market_assets,news_assets), axis=0))

In [8]:
#Number of assets found in both datasets
print('Number of assets found in both datasets',len(np.intersect1d(market_assets,news_assets)))

Number of assets found in both datasets 2805


In [9]:
#Total number of assets mentioned
print('Total number of assets mentioned',len(assets))

Total number of assets mentioned 10609


In [10]:
#Total number of assets in the market
print('Total number of assets in the market',len(market_assets))

Total number of assets in the market 2860


In [11]:
#Total number of assets in the news
print('Total number of assets in the news',len(news_assets))

Total number of assets in the news 10554


As observed, many of the assets appears on the news but not all of them appears on the market data. Using asset embeddings instead of a fixed asset code should allow the system to discover more the relationship between the assets through news and market signals.

In [12]:
assets

array(['000030.KS', '0005.HK', '005490.KS', ..., 'ZYNE.OQ', 'ZZ.N',
       'ZZC.N'], dtype=object)

In [13]:
pivot=market_train_df.pivot(index='time',columns='assetCode',values='returnsClosePrevMktres10').reindex_axis(assets, axis=1).fillna(0)

C:\Users\kin.poon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """Entry point for launching an IPython kernel.


In [14]:
def PCA_embedding(df,timeStart,timeEnd,components):
    '''
    Do the time cutting and calculate the pca. 
    The index of the dataframe must be datetime
    '''
    import numpy as np
    from sklearn.decomposition import PCA
    if(not isinstance(pivot.index, pd.DatetimeIndex)):
        df.index = pd.to_datetime(df.index )
    T_s=pd.Timestamp(timeStart)
    T_e=pd.Timestamp(timeEnd)
    df_T=df[T_s:T_e]
    pca = PCA(n_components=components)
    pca.fit(df_T)
    centre=list(df_T.mean())
    return pca,centre

In [15]:
tf.reset_default_graph()

AttributeError: module 'tensorflow' has no attribute 'reset_default_graph'

In [16]:
embedded_dim=30
assets_num=len(assets)
embedding_shape=(assets_num,embedded_dim)

In [149]:
pca,centre=PCA_embedding(pivot,'2013-1-1','2013-12-30',30)
embedding_weight_init=np.transpose(pca.components_)
embedding_bias_init=np.matmul(centre,embedding_weight_init)

In [150]:
np.shape(embedding_weight_init)

(10609, 30)

In [151]:
embedding=tf.get_variable('asset_embedding',shape=embedding_shape,dtype=tf.float64)
embedding_W=tf.get_variable('asset_weights',initializer=embedding_weight_init,dtype=tf.float64)
embedding_b=tf.get_variable('asset_bias',initializer=embedding_bias_init,dtype=tf.float64)

embedding_V=tf.get_variable('asset_gateWeights',initializer=tf.zeros_initializer,shape=(embedded_dim,assets_num),dtype=tf.float64)
embedding_c=tf.get_variable('asset_gateBias',initializer=tf.ones_initializer,shape=(embedded_dim,assets_num),dtype=tf.float64)

embedded_code=tf.multiply(
    tf.add(
        tf.matmul(embedding,embedding_W),embedding_b),
    tf.add(
        tf.nn.sigmoid(tf.matmul(embedding,embedding_V),embedding_c))
)

ValueError: Dimensions must be equal, but are 30 and 10609 for 'MatMul' (op: 'MatMul') with input shapes: [10609,30], [10609,30].

array([[ 1.67571185e-03, -9.63768962e-18, -2.37400476e-03, ...,
        -7.80981503e-03, -0.00000000e+00,  3.63670640e-02],
       [ 2.50574892e-04, -7.06640193e-17,  1.87747514e-03, ...,
        -3.58260816e-03,  0.00000000e+00, -2.24981168e-02],
       [-2.22077617e-02,  1.71027975e-16, -1.18437865e-04, ...,
         2.57458893e-03, -0.00000000e+00,  2.15119324e-02],
       ...,
       [-8.80399051e-03,  2.76705658e-16, -6.74032219e-03, ...,
         1.61098155e-02,  0.00000000e+00, -2.10604387e-02],
       [ 8.29686811e-03,  3.85268785e-17,  2.60499307e-02, ...,
        -8.70926846e-03, -0.00000000e+00, -3.42163782e-02],
       [ 6.54405942e-03, -4.37907164e-18, -9.84918212e-03, ...,
         2.30639292e-02, -0.00000000e+00, -5.03609494e-03]])